In [3]:
import sqlite3
import csv
import os
from cities.utils.cleaning_utils import find_repo_root, standardize_and_scale
import pandas as pd
from cities.utils.data_grabber import list_available_features, DataGrabber
from typing import List
import time
import numpy as np
repo_root = find_repo_root()


In [5]:
# Functions to read and write to a SQLite database
def add_variable_table_to_db(variable_name, database_path): 
    # check data format
    df = pd.read_csv(os.path.join(repo_root, 'data', 'processed', variable_name + '_long.csv'))
    cols = df.columns
    assert len(cols) == 4, 'Data should have 4 columns: GeoFIPS, GeoName, Year/Category, Value'
    assert cols[0] == 'GeoFIPS' , 'First column should be GeoFIPS'
    assert cols[1] == 'GeoName' , 'Second column should be GeoName'
    col_name_category = cols[2] # sometimes this is year, sometimes category
    col_name_value = cols[3] # usually this is value, sometimes more specific

    if df[col_name_category].dtype == 'int': # TODO: currently in dataset, yr is sometimes str, sometimes int
        col_cat_type = 'INTEGER'
    else:
        col_cat_type = 'TEXT'

    # Step 1: Connect to a SQLite database
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    # Step 2: Drop the existing table if it exists
    cursor.execute('DROP TABLE IF EXISTS ' + variable_name)

    # Create the table with a composite primary key
    cursor.execute(
        'CREATE TABLE IF NOT EXISTS ' + variable_name + ' ('
            '''GeoFIPS INTEGER,
            GeoName TEXT,'''
            + col_name_category + ' ' + col_cat_type + ','
            + col_name_value + ' REAL,'
            'PRIMARY KEY (GeoFIPS, ' + col_name_category + ')'
            ')'
    )
    
    # Use pandas to insert data
    df.to_sql(variable_name, conn, if_exists='replace', index=False)
    
    # Close the connection
    conn.commit()
    conn.close()

def read_variable_table_from_db(variable_name, database_path):
    # Step 1: Connect to a SQLite database
    conn = sqlite3.connect(database_path)
    cursor = conn.cursor()

    # Step 2: Read the entire table into a pandas DataFrame
    df = pd.read_sql_query('SELECT * FROM ' + variable_name, conn)

    # Step 4: Close the connection
    conn.close()

    return df

In [6]:
#Add a single variable (GDP) to the database, and read it back
variable_name = 'gdp'
database_path = os.path.join(repo_root, 'data', 'us_counties.db')

add_variable_table_to_db(variable_name, database_path)
df = read_variable_table_from_db(variable_name, database_path)
df.head()


,GeoFIPS,GeoName,Year,Value
0,1001,"Autauga, AL",2001,59.839
1,1003,"Baldwin, AL",2001,73.853
2,1005,"Barbour, AL",2001,113.864
3,1007,"Bibb, AL",2001,80.443
4,1009,"Blount, AL",2001,92.104


In [7]:
# ADD ALL VARIABLES TO THE DB
# iterate over all variables
feature_list = list_available_features()
database_path = os.path.join(repo_root, 'data', 'us_counties.db')
for variable_name in feature_list:
    print(variable_name)
    add_variable_table_to_db(variable_name, database_path)
    df = read_variable_table_from_db(variable_name, database_path)
    print(df.head())


# Note, data/processed/industry_long.csv has a slight bug: 'Year' is a column name, when it should be 'Category'

spending_HHS
   GeoFIPS      GeoName  Year  total_obligated_amount
0     1001  Autauga, AL  2018                     0.0
1     1003  Baldwin, AL  2010              49158971.0
2     1003  Baldwin, AL  2011              23130080.0
3     1003  Baldwin, AL  2012              23130080.0
4     1003  Baldwin, AL  2013              23130080.0
industry_mining_total
   GeoFIPS      GeoName  Year  Value
0     1001  Autauga, AL  2010   66.0
1     1001  Autauga, AL  2011   56.0
2     1001  Autauga, AL  2012   84.0
3     1001  Autauga, AL  2013   92.0
4     1001  Autauga, AL  2014  106.0
spending_commerce
   GeoFIPS      GeoName  Year  total_obligated_amount
0     1001  Autauga, AL  2017               2364930.0
1     1003  Baldwin, AL  2011               1274298.0
2     1003  Baldwin, AL  2013                186580.0
3     1003  Baldwin, AL  2014                105434.0
4     1003  Baldwin, AL  2015                105434.0
spending_transportation
   GeoFIPS      GeoName  Year  total_obligated_amount

In [12]:
class DataGrabber_FROM_DB:
    def __init__(self):
        self.repo_root = find_repo_root()
        self.database_path = os.path.join(self.repo_root, 'data', 'us_counties.db')
        self.wide = {}
        self.std_wide = {}
        self.long = {}
        self.std_long = {}

    def _fetch_data(self, table_name: str) -> pd.DataFrame:
        if table_name not in self.long:
            conn = sqlite3.connect(self.database_path)
            self.long[table_name] = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
            conn.close()
        return self.long[table_name]

    def _convert_to_wide(self, df: pd.DataFrame, index_cols: List[str], columns_col: str, values_col: str) -> pd.DataFrame:
        # Pivot the DataFrame
        df_wide = df.pivot(index=index_cols, columns=columns_col, values=values_col)

        # Reset the index and fill missing values with 0.0
        df_wide.reset_index(inplace=True)

        # Fill NaN values with 0.0
        df_wide.fillna(0.0, inplace=True)

        # Remove the name of the index column if it exists
        df_wide.index.name = None

        # Ensure the column names do not retain the name of 'columns_col'
        df_wide.columns.name = None

        return df_wide
    def get_features_wide(self, features: List[str]) -> None:
        for feature in features:
            if feature not in self.wide:
                df = self._fetch_data(feature)
                columns_col = df.columns[-2]
                values_col = df.columns[-1]
                self.wide[feature] = self._convert_to_wide(df, ['GeoFIPS', 'GeoName'], columns_col, values_col)

    def get_features_std_wide(self, features: List[str]) -> None:
        for feature in features:
            if feature not in self.std_wide:
                if feature not in self.wide:
                    self.get_features_wide([feature])
                self.std_wide[feature] = standardize_and_scale(self.wide[feature])

    def get_features_long(self, features: List[str]) -> None:
        for feature in features:
            if feature not in self.long:
                self._fetch_data(feature)

    def get_features_std_long(self, features: List[str]) -> None:
        for feature in features:
            if feature not in self.std_long:
                if feature not in self.std_wide:
                    self.get_features_std_wide([feature])

                # Retrieve the standardized wide format data
                df_wide = self.std_wide[feature]
                df = self._fetch_data(feature)
                columns_col = df.columns[-2]
                values_col = df.columns[-1]
                df_long = pd.melt(df_wide, id_vars=["GeoFIPS", "GeoName"], var_name=columns_col, value_name=values_col)

                # Store the melted DataFrame
                self.std_long[feature] = df_long


In [13]:
# Test that the two DataGrabbers give the same data, and compare speed
features = list_available_features()

# Original DataGrabber (CSV)
start_time_csv = time.time()
data_grabber_csv = DataGrabber()
data_grabber_csv.get_features_long(features)  
data_grabber_csv.get_features_wide(features)  
data_grabber_csv.get_features_std_wide(features)  
data_grabber_csv.get_features_std_long(features)  
end_time_csv = time.time()

# New DataGrabber (DB)
start_time_db = time.time()
data_grabber_db = DataGrabber_FROM_DB()
data_grabber_db.get_features_long(features)  
data_grabber_db.get_features_wide(features)
data_grabber_db.get_features_std_wide(features)
data_grabber_db.get_features_std_long(features)
end_time_db = time.time()

# Compare results (example for one feature)
for feature in features:
      print(feature)
      
      # in the existing dataset, the years are inconsistent types (Str vs int). convert all to str
      data_grabber_db.wide[feature].columns = data_grabber_db.wide[feature].columns.map(str)
      data_grabber_csv.wide[feature].columns = data_grabber_csv.wide[feature].columns.map(str)
      # make the columns the same order
      data_grabber_db.wide[feature] = data_grabber_db.wide[feature][data_grabber_csv.wide[feature].columns]
      print( "    Are the results identical?: long: ", data_grabber_csv.long[feature].equals(data_grabber_db.long[feature]), 
            ' | wide: ', data_grabber_csv.wide[feature].equals(data_grabber_db.wide[feature]), 
            " | std_long: ", data_grabber_csv.std_long[feature].equals(data_grabber_db.std_long[feature]),
            ' | std_wide: ', data_grabber_csv.std_wide[feature].equals(data_grabber_db.std_wide[feature]))
      # # data_grabber_db.std_wide[feature] = data_grabber_db.std_wide[feature][data_grabber_csv.std_wide[feature].columns]
      # print('std_long: ', data_grabber_csv.std_long[feature].equals(data_grabber_db.std_long[feature]))
      # print(data_grabber_csv.std_long[feature].head())
      # print(data_grabber_db.std_long[feature].head())
      # print()

# Print performance
print("CSV DataGrabber Time:", end_time_csv - start_time_csv)
print("DB DataGrabber Time:", end_time_db - start_time_db)
# Note: investigate spending variables -- mismatch wide vs long

ValueError: value_name (2021) cannot match an element in the DataFrame columns.

In [11]:
data_grabber_csv.std_long['unemployment_rate'].head()

,GeoFIPS,GeoName,Year,Value
0,1001,"Autauga County, AL",1990,0.020347
1,1003,"Baldwin County, AL",1990,-0.047713
2,1005,"Barbour County, AL",1990,0.099469
3,1007,"Bibb County, AL",1990,0.171851
4,1009,"Blount County, AL",1990,0.014674


# Next steps

# Design decisions to make 
- Years are sometimes str, sometimes int. What do we want?
- What relations do we want to build in between tables? (i.e link by GeoFIPS + Year)
- I wrote 